In [183]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

In [184]:
data = pd.read_csv('X_train.csv')
train, test = train_test_split(data,random_state = 1,test_size = 0.2)
y_test = test['reting']
y = train['reting']
index = train.shape[0]
test['reting'] = -1
#разделили на трейн и тест, теперь опять ссоединим вместе, чтобы сразу чистить и генерить фичи на всем датасете
df = train.append(test,ignore_index=True)

In [185]:
#можно как-то использовать колонку property, но так как это отборочный этап, и что-то сложное делать не будем, просто удалим колонку
del df['property']
# userName зашифрован в MD5, можно конечно попробовать расшифровать процентов 40% ников, по никам нагенерить признаки, вроди пола
#и тд но мы это тоже делать не будем

In [186]:
#нагенерим фичи, связанные с датой
df['date'] = pd.to_datetime(df['date'])
df['day'] = df['date'].apply(lambda x: x.day)
df['weekday'] = df['date'].apply(lambda x: x.weekday())
df['month'] = df['date'].apply(lambda x: x.month)
df['year'] = df['date'].apply(lambda x: x.year)
del df['date']

In [187]:
df['commentNegative'] = df['commentNegative'].fillna('')
df['commentPositive'] = df['commentPositive'].fillna('')
df['commentPositive_len'] = df['commentPositive'].str.len()
df['commentNegative_len'] = df['commentNegative'].str.len()
df['comment_len'] = df['comment'].str.len()

In [188]:
#так как commentNegative и commentNegative во многих отзывах пропущены, то просто склеим их к коменту и удалим
#в роли флага, был ли негативный или позитинвый коммент, будет выступать commentNegative_len и commentPositive_len
df['comment'] = df['comment'] + ' œ ' + df['commentPositive'] + ' å ' +  df['commentNegative']

In [189]:
df['comment_len_all'] = df['comment'].str.len()

In [190]:
del df['commentNegative']
del df['commentPositive']

In [191]:
def average(col,alpha = 10,y_target = 'reting'):
    ans = pd.Series(index = df.index)#создали пустой массив, с индексами,и размерностью df
    X_train = df[df['reting'] != -1][[col,y_target]]#создали новый датафрейм из друх колонок:колонки col and y_target, берем только трейн данные
    X_test = df[df['reting'] == -1][col]#копируем колонку col тест данных
    mean_by_col = X_train.groupby(col).mean()[y_target]#для каждого уникального значения колонки col найдем средний таргет
    counts = X_train[col].value_counts()#подсчитаем количество каждого уникального значения col
    mean = X_train[y_target].mean()# запомним глобальное среднее таргета
    ans_test = X_test.apply(lambda x: (mean_by_col[x]*counts[x] + alpha*mean) / (counts[x] + alpha)  if x in counts.index else mean)
    #строчка выше вычисляет формулу из видео
    ans[ans_test.index] = ans_test#на нужные инлексы запишем наш полученный результат.
    #это мы зная весь трейн, вычислили сркднее по таргету для теста. Теперь надо получить также среднее для трейна, но чтобы не переобучаться
    #мы должны использовать кросс валидацию.Перемешаем и Разобьем трецн на 5 частей. на 4 частях будем обучатьсч, а на пятой предсказывать
    kf = KFold(n_splits=5,random_state=42,shuffle = True)
    for train_i,test_i in kf.split(X_train):
        #train_i - индексы, которые попали в первые 4 части и на них будем обучаться
        #test_i -индексы для которых будем делать предсказания
        #теперь делаем все тоже самое, что и выше, когда мы предсказывали среднее для теста по трейну
        X = X_train.iloc[train_i]
        X_t = X_train.iloc[test_i,0]
        mean_by_col = X.groupby(col).mean()[y_target]
        counts = X[col].value_counts()
        mean = X[y_target].mean()
        ans_test = X_t.apply(lambda x: (mean_by_col[x]*counts[x] + alpha*mean) / (counts[x] + alpha)  if x in counts.index else mean)
        ans[ans_test.index] = ans_test
    return ans

In [192]:
#как учил Семенов, делаем mean target для всех колонок. Двойную кросс валидацию делать не будем, так как и так сойдет =) 

In [193]:
col_to_mean_target = ['sku','categoryLevel1Id','categoryLevel2Id','brandId','userName','day','weekday','month',
                     'year','commentPositive_len','commentNegative_len','comment_len','comment_len_all']

In [194]:
for i in col_to_mean_target:
    df[i + '_mean_target'] = average(i)

Теперь еще одна сложная штука для понимания, но простая в реализации. Это субъективное восприятие товара человеком. Бывают люди гурманы, которые считают что все товары говно, и всегда ставят оценку к товару ниже, чем средняя оценка. А есть наоборот, люди, которым все товары нравятся и они ставят завышенную оценку. Создадим новую переменную: разницу между оценкой, которую поставил человек и средней оценкой товара

In [195]:
df['tmp'] = df['reting'] - df['categoryLevel2Id_mean_target']

И теперь применим среднее по таргету для кажого человека, при этом таргетом у нас как раз будет tmp переменная. Что мы получили? Получили на сколько в среднем отзыв человека ниже/выше среднего отзыва о товаре. Ну и добавим новую фичу: предпологаемая оценка товара человеком.

In [196]:
df['delta_score'] = average('userName',alpha=1,y_target='tmp')
df['predict'] = df['categoryLevel2Id_mean_target'] + df['delta_score']

Теперь займемся самими текстами. Не будем использовать модные word2vec, нейронки, LSTM, а возьмем добрый, проверенный временем мешок слов tfidf

Так как организарты не обозначили конкретную метрику, то будем использовать MSE, так как у нас отзывы упорядочены. В конце будем просто округлять до целого.

с помощью GridSearchCV найдем оптимальные параметры для TfidfVectorizer

In [197]:
%%time
for i in range(1,4):
    tf = TfidfVectorizer(ngram_range=(1,i))
    summary = tf.fit_transform(df['comment'])
    grid ={'alpha':[0.1]}
    gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=-1)
    gr.fit(summary[:index],y)
    print i,gr.grid_scores_

1 [mean: -1.07582, std: 0.02202, params: {'alpha': 0.1}]
2 [mean: -0.93290, std: 0.02947, params: {'alpha': 0.1}]
3 [mean: -1.00315, std: 0.03014, params: {'alpha': 0.1}]
CPU times: user 29.6 s, sys: 2.01 s, total: 31.7 s
Wall time: 37.7 s


In [198]:
%%time
for i in range(100000,1100000,100000):
    tf = TfidfVectorizer(ngram_range=(1,2),max_features =  int(i))
    summary = tf.fit_transform(df['comment'])
    grid ={'alpha':[0.1]}
    gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=-1)
    gr.fit(summary[:index],y)
    print i,gr.grid_scores_

100000 [mean: -0.89924, std: 0.02366, params: {'alpha': 0.1}]
200000 [mean: -0.90170, std: 0.02656, params: {'alpha': 0.1}]
300000 [mean: -0.91243, std: 0.02804, params: {'alpha': 0.1}]
400000 [mean: -0.92382, std: 0.02876, params: {'alpha': 0.1}]
500000 [mean: -0.93288, std: 0.02944, params: {'alpha': 0.1}]
600000 [mean: -0.93290, std: 0.02946, params: {'alpha': 0.1}]
700000 [mean: -0.93286, std: 0.02945, params: {'alpha': 0.1}]
800000 [mean: -0.93286, std: 0.02947, params: {'alpha': 0.1}]
900000 [mean: -0.93289, std: 0.02945, params: {'alpha': 0.1}]
1000000 [mean: -0.93288, std: 0.02946, params: {'alpha': 0.1}]
CPU times: user 1min 31s, sys: 4.95 s, total: 1min 36s
Wall time: 1min 53s


In [199]:
%%time
for i in range(10000,110000,10000):
    tf = TfidfVectorizer(ngram_range=(1,2),max_features =  int(i))
    summary = tf.fit_transform(df['comment'])
    grid ={'alpha':[0.1]}
    gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=-1)
    gr.fit(summary[:index],y)
    print i,gr.grid_scores_

10000 [mean: -1.19361, std: 0.02797, params: {'alpha': 0.1}]
20000 [mean: -1.01777, std: 0.02566, params: {'alpha': 0.1}]
30000 [mean: -0.96122, std: 0.02379, params: {'alpha': 0.1}]
40000 [mean: -0.93686, std: 0.02254, params: {'alpha': 0.1}]
50000 [mean: -0.91983, std: 0.02415, params: {'alpha': 0.1}]
60000 [mean: -0.91200, std: 0.02460, params: {'alpha': 0.1}]
70000 [mean: -0.90527, std: 0.02445, params: {'alpha': 0.1}]
80000 [mean: -0.90033, std: 0.02364, params: {'alpha': 0.1}]
90000 [mean: -0.90027, std: 0.02389, params: {'alpha': 0.1}]
100000 [mean: -0.89925, std: 0.02365, params: {'alpha': 0.1}]
CPU times: user 1min 17s, sys: 4.07 s, total: 1min 21s
Wall time: 1min 32s


Нашли оптимальные параметры для  tfidf: ngram_range=(1,2),max_features = 100000

Мы создали новые фичи. Возьмем фичи, которые увеличивают score. И отнормируем их, так как мы будем использовать линейную регрессию с l2 регуляризацией, ее также называют Ridge.

In [203]:
data = df[['delta_score','predict','comment_len_all_mean_target','comment_len_mean_target',
    'commentNegative_len_mean_target','commentPositive_len_mean_target']]

In [204]:
ss = StandardScaler()
data_s=ss.fit_transform(data)

In [205]:
X_test = hstack([summary[index:,],data_s[index:,]])
X_train = hstack([summary[:index,],data_s[:index,]])

Для проверки результатов, отбора фич и настройки параметра alpha, я использовал кросс-валидацию. Для этого использовал GridSearchCV. Так как данные хорошо были перемешаны, то достаточно было использовать всего два фолда, т е разбивать трейн на две части, на одной обучать, а на другой проверять.

In [215]:
%%time
grid ={'alpha':np.linspace(0.000000000001,0.1,15)}
gr = GridSearchCV(Ridge(),grid,cv = 2,scoring='neg_mean_squared_error',n_jobs=-1)
gr.fit(X_train,y)

CPU times: user 10.4 s, sys: 640 ms, total: 11.1 s
Wall time: 2min 18s


In [216]:
gr.grid_scores_

[mean: -0.89177, std: 0.02741, params: {'alpha': 9.9999999999999998e-13},
 mean: -0.89212, std: 0.02734, params: {'alpha': 0.007142857143785715},
 mean: -0.89240, std: 0.02750, params: {'alpha': 0.014285714286571429},
 mean: -0.89274, std: 0.02742, params: {'alpha': 0.021428571429357141},
 mean: -0.89309, std: 0.02736, params: {'alpha': 0.028571428572142857},
 mean: -0.89355, std: 0.02740, params: {'alpha': 0.035714285714928573},
 mean: -0.89395, std: 0.02725, params: {'alpha': 0.042857142857714282},
 mean: -0.89433, std: 0.02737, params: {'alpha': 0.050000000000499999},
 mean: -0.89466, std: 0.02746, params: {'alpha': 0.057142857143285715},
 mean: -0.89501, std: 0.02738, params: {'alpha': 0.064285714286071438},
 mean: -0.89556, std: 0.02741, params: {'alpha': 0.071428571428857154},
 mean: -0.89604, std: 0.02745, params: {'alpha': 0.078571428571642871},
 mean: -0.89630, std: 0.02747, params: {'alpha': 0.085714285714428573},
 mean: -0.89666, std: 0.02738, params: {'alpha': 0.09285714285

Тут можно много что еще делать, юзать xgb, нейронку, потом запилить стекинг, но так как это тестовое задание, и в github вбив в поиск mvideo, можно посмотреть решения других участников, то понятно, что участников не так много, так что можно остановиться на этом. Теперь просто предскажем нашей моделью тестовые данные.

In [217]:
rg = Ridge(alpha=9.9999999999999998e-13)
rg.fit(X_train,y)

Ridge(alpha=1e-12, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [218]:
ans = rg.predict(X_test)

In [220]:
from sklearn import metrics

In [221]:
#MSE
metrics.mean_squared_error(y_test,ans)

0.85679125311542081

array([ 3.38185203,  4.84287723,  5.11438313, ...,  3.17035521,
        3.38514497,  3.77308691])